# Quality Measure Portal

## special nodes
 - Pass id = 99
 - Fail id = 100
 - Exclude = 101

In [1]:
import json
import numpy as np

In [184]:
class QMBaseNode:

    # Read node from json and initialize required data
    # Include evaluation method, required paths, expected value(s)
    # and traversal info
    # Override with derived classes as needed.  Especially end states.

    def __init__(self, tree, nodeInfo):
        self.tree = tree
        self.passNodeId = nodeInfo['passId']
        self.failNodeId = nodeInfo['failId']

        self.valuePath = nodeInfo['path']
        self.expectedValue = nodeInfo['expectedValue']


    # Called by other nodes, or the tree, to evaluate the node and pass
    # execution to the next node
    
    def evaluate(self, patientData):
        if self.evalExpression(self.valuePath, patientData) == self.expectedValue:
            return self.tree.getNode(self.passNodeId).evaluate(patientData)
        else:
            return self.tree.getNode(self.failNodeId).evaluate(patientData)

        
        
    def evalExpression(self, valuePath, patientData):
        path = valuePath.split('.')
        nested_value = patientData
        
        for feature in path:
            if feature in nested_value:
                nested_value = nested_value.get(feature)
            else:
                return None
        return nested_value

In [185]:
class QMEndNode(QMBaseNode):
    def __init__(self, tree, nodeInfo):
        self.tree = tree
        self.valuePath = ""
        self.endState = nodeInfo['result']

    def evaluate(self, patientData=None):
        return str(self.endState)

In [186]:
class QMTree:
    dt_json = None
    def __init__(self):
        self.nodes = dict()

    def readTreeFile(self, path):
        with open(f'{path}.json', 'r') as f:
            self.tree = json.load(f)
            
            
        for item in self.tree['nodes']:
            if item['type']=="base":
                self.nodes[item['featureID']] = QMBaseNode(self, item)
            else:
                self.nodes[item['featureID']] = QMEndNode(self, item)

        
    def addNode(self, node):
        pass
    
    def removeNode(self, node):
        pass
    
    def getNode(self, id):
        try:
            return self.nodes[id]
        except:
            print('Warning: no such node with id ' , id)
            return None
        
    def evaluate(self, startId, patientData):
        return self.getNode(startId).evaluate(patientData)

         
           
    def get_next(self, next_node, dt_json):
        '''search for the next item'''
        print('Next node is : ', next_node)
        for node_dict in dt_json:
            if node_dict['featureID']==next_node:
                return node_dict



In [188]:
myTree = QMTree()
myTree.readTreeFile("DT2")


with open('Data/data.json', 'r') as f:
    dataset = json.load(f)


print([myTree.evaluate("1", dataset[patient]) for patient in range(len(dataset))])

['exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'fail', 'fail', 'fail', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'fail', 'fail', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'exclude', 'e